### Question 2

## In this assignment we have used web scrapping and created a dataset which has 3 divisions of health, climate and business.

#### 1.Create your own dataset for text classification. It should contain at least 1000 words in total and at least two categories with at least 100 examples per category.

In [214]:
import pandas as pd
df=pd.read_csv('dataset_aml.csv')

In [224]:
df.replace('health',0,inplace=True)
df.replace('climate',1,inplace=True)
df.replace('business',2,inplace=True)

In [225]:
from datasets import Dataset

#### 2. Split the dataset into training (at least 160examples) and test (at least 40 examples) sets. 

In [226]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [227]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [228]:
train_data.shape

(360, 3)

In [229]:
test_data.shape

(90, 3)

In [230]:
train_data=train_data.iloc[:,1:]
test_data=test_data.iloc[:,1:]

In [231]:
train_data_tkn = Dataset.from_pandas(train_data)
test_data_tkn = Dataset.from_pandas(test_data)

In [233]:
import transformers
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_train = tokenizer(train_data_tkn["text"], return_tensors="np", padding=True)
tokenized_test = tokenizer(test_data_tkn["text"], return_tensors="np", padding=True)

tokenized_train = dict(tokenized_train)
tokenized_test = dict(tokenized_test)

labels_train = np.array(train_data_tkn["label"])
labels_test = np.array(test_data_tkn["label"])

#### 3. Fine tune a pretrained language model capable of generating text (e.g., GPT) that you can take from the Hugging Face Transformers library with the dataset your created (this tutorial could be very helpful: https://huggingface.co/docs/transformers/training). Report the test accuracy. Discuss what could be done to improve accuracy. 

In [235]:

from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf


## Model Definition
model = TFAutoModelForSequenceClassification.from_pretrained("BSC-TeMU/roberta-base-bne", from_pt=True, num_labels=3)

## Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, 
              loss=loss,
              metrics=metric) 

## Fitting the data 
history = model.fit(tokenized_train, labels_train, epochs=5, batch_size=64)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/5
6/6 [==============================] - 108s 14s/step - loss: 1.1026 - sparse_categorical_accuracy: 0.3194
Epoch 2/5
6/6 [==============================] - 81s 13s/step - loss: 1.1007 - sparse_categorical_accuracy: 0.3417
Epoch 3/5
6/6 [==============================] - 80s 13s/step - loss: 1.0976 - sparse_categorical_accuracy: 0.3583
Epoch 4/5
6/6 [==============================] - 83s 14s/step - loss: 1.0883 - sparse_categorical_accuracy: 0.3333
Epoch 5/5
6/6 [==============================] - 80s 13s/step - loss: 1.1023 - sparse_categorical_accuracy: 0.3139


In [236]:
predictions= model.predict(tokenized_test)
predictions

3/3 [==============================] - 9s 2s/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 0.00409388, -0.17499132,  0.10081658],
       [ 0.00410101, -0.17500289,  0.10081441],
       [ 0.0041034 , -0.17501122,  0.10081749],
       [ 0.00409702, -0.17500857,  0.10081347],
       [ 0.00409907, -0.17501134,  0.10081636],
       [ 0.00409364, -0.17500266,  0.10081191],
       [ 0.00409187, -0.17498949,  0.10081265],
       [ 0.00409186, -0.1749943 ,  0.10081364],
       [ 0.00409374, -0.1749948 ,  0.10080694],
       [ 0.00409551, -0.17500436,  0.1008113 ],
       [ 0.00408841, -0.17498237,  0.10081255],
       [ 0.00410164, -0.17500797,  0.10081722],
       [ 0.00409571, -0.17500158,  0.10081238],
       [ 0.00409209, -0.17499475,  0.10081135],
       [ 0.00409792, -0.17501625,  0.10081596],
       [ 0.00409924, -0.17500757,  0.10081818],
       [ 0.0041016 , -0.17501405,  0.10081898],
       [ 0.00409707, -0.1750015 ,  0.10081585],
       [ 0.00408811, -0.1749983 ,  0.10081268],
       [ 0.00409639, -0.17499745,  0.100813

In [237]:
model.evaluate(tokenized_test,labels_test)

3/3 [==============================] - 8s 2s/step - loss: 1.1214 - sparse_categorical_accuracy: 0.2667


[1.12144136428833, 0.2666666805744171]

In [239]:

from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf


## Model Definition
model2 = TFAutoModelForSequenceClassification.from_pretrained("BSC-TeMU/roberta-base-bne", from_pt=True, num_labels=3)

## Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
model2.compile(optimizer=optimizer, 
              loss=loss,
              metrics=metric) 

## Fitting the data 
history2 = model2.fit(tokenized_train, labels_train, epochs=5, batch_size=64)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/5
6/6 [==============================] - 110s 13s/step - loss: 1.1634 - sparse_categorical_accuracy: 0.3500
Epoch 2/5
6/6 [==============================] - 81s 13s/step - loss: 1.1348 - sparse_categorical_accuracy: 0.3194
Epoch 3/5
6/6 [==============================] - 82s 13s/step - loss: 1.1135 - sparse_categorical_accuracy: 0.3083
Epoch 4/5
6/6 [==============================] - 80s 13s/step - loss: 1.1232 - sparse_categorical_accuracy: 0.2806
Epoch 5/5
6/6 [==============================] - 83s 14s/step - loss: 1.1228 - sparse_categorical_accuracy: 0.3472


In [240]:
model2.evaluate(tokenized_test,labels_test)

3/3 [==============================] - 8s 2s/step - loss: 1.1146 - sparse_categorical_accuracy: 0.3444


[1.1145516633987427, 0.3444444537162781]

There are multiple strategies to enhance the accuracy of a pre-trained language model like BSC-TeMU/roberta-base-bne on a text classification dataset. One method involves fine-tuning the model on the specific task using the dataset, where adjustments to hyperparameters and optimizing the loss function are made. Another effective approach is to employ data augmentation techniques such as back-translation or synonym replacement to diversify the training data. Furthermore, utilizing an ensemble of models or training the model with a larger corpus of data can elevate its accuracy. Lastly, regularization techniques like dropout or early stopping aid in improving the model's generalization performance.